# Moving Averages CINGULATE region

# 1) Imports

In [ ]:
import os
import glob
import sys
import json

p = os.path.abspath('../')
if p not in sys.path:
    sys.path.append(p)

Error: Kernel is dead

In [ ]:
import moving_averages as ma
import colorado as cld
import dico_toolbox as dtx
from tqdm import tqdm

import pandas as pd
import numpy as np
from scipy.spatial import distance

import plotly.graph_objects as go
import pickle
import matplotlib.pyplot as plt

from soma import aims

import torch

from sklearn.cluster import KMeans, SpectralClustering, AffinityPropagation
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.manifold import TSNE

import matplotlib.cm as cm

from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from omegaconf import OmegaConf, DictConfig

# 2) Functions and datasets

In [ ]:
def closest_distance(centroid, df):
    """Returns closest point to centroid of a given cluster
    /!\ central_point is part of the defined set of points
    Args:
        IN: centroid: tuple of coordinates of centroid
            df: pandas.DataFrame with index corresponding to subjects and as many columns as coordinates
        OUT: central_point: subject ID corresponding to closest point to the centroid
    """
    # df must have as many columns as dimensions to consider
    distances = {}
    # Get distances to centroid for each point
    for sub in list(df.index):
        pos = [df.loc[sub][k] for k in range(0, len(df.columns))]
        distances[sub] = distance.euclidean(pos, centroid)
    # Get closest point to centroid
    central_point = min(distances, key=distances.get)
    return central_point

In [ ]:
def mscatter(x, y, ax=None, m=None, **kw):
    if not ax:
        ax = plt.gca()
    sc = ax.scatter(x, y, **kw)
    if (m is not None) and (len(m) == len(x)):
        paths = []
        for marker in m:
            if isinstance(marker, mmarkers.MarkerStyle):
                marker_obj = marker
            else:
                marker_obj = mmarkers.MarkerStyle(marker)
            path = marker_obj.get_path().transformed(
                marker_obj.get_transform())
            paths.append(path)
        sc.set_paths(paths)
    return sc

In [ ]:
path = '/neurospin/dico/data/deep_folding/current/crops/CINGULATE/mask/sulcus_based/2mm/centered_combined/hcp/Rcrops/'

In [ ]:
if torch.cuda.is_available():
    device = "cuda:0"
print(device)

In [ ]:
# We load the mebeddings
result_dir = '/host/volatile/jc225751/Runs/28_cingulate_back_to_2mm/Output/t-0.1/n-004_o-4_2nde_half'
train_val_embeddings_file = f"{result_dir}/train_val_embeddings.pt"
embeddings = torch.load(train_val_embeddings_file)
print(embeddings.shape)

In [ ]:
# We load the labels
with open(f"{result_dir}/train_val_filenames.json", 'r') as f:
    subject_labels = json.load(f)
print(subject_labels[0])    

# 3) T-SNE and different clusterings

In [ ]:
tsne = TSNE(n_components=2, perplexity=30, init='pca', random_state=50)
X_tsne = tsne.fit_transform(embeddings)

In [ ]:
fig, ax = plt.subplots(1)
mscatter(X_tsne[:, 0], X_tsne[:, 1], s=20, ax=ax)
plt.show()


In [ ]:
X_tsne.shape 

In [ ]:
af = AffinityPropagation().fit(X_tsne)
cluster_labels_ini = af.labels_
initial_centers = af.cluster_centers_indices_
n_clusters_ = len(initial_centers)
while n_clusters_ > 5:
    af = AffinityPropagation().fit(X_tsne[af.cluster_centers_indices_])
    cluster_labels_af = af.labels_
    x_cluster_label_af = af.predict(X_tsne)
    n_clusters_ = len(af.cluster_centers_indices_)
    print(n_clusters_)

print(f"Average silhouette score: {metrics.silhouette_score(X_tsne, x_cluster_label_af)}")

In [ ]:
fig, ax = plt.subplots(1)
mscatter(X_tsne[:, 0], X_tsne[:, 1], c=x_cluster_label_af, s=20, ax=ax)
plt.show()

In [ ]:
kmeans = KMeans(n_clusters=10, random_state=0).fit(X_tsne)
cluster_labels_kmeans = kmeans.labels_
x_cluster_label_kmeans = kmeans.predict(X_tsne)
clusters_centroids_kmeans = kmeans.cluster_centers_
print(f"clusters centroids = {clusters_centroids_kmeans.shape}")
print(f"Average silhouette score: {metrics.silhouette_score(X_tsne, cluster_labels_kmeans)}")


In [ ]:
fig, ax = plt.subplots(1)
mscatter(X_tsne[:, 0], X_tsne[:, 1], c=x_cluster_label_kmeans, s=20, ax=ax)
plt.show()

In [ ]:
dbscan = DBSCAN(eps=2).fit(X_tsne)
cluster_labels_dbscan = dbscan.labels_
x_cluster_label_dbscan = dbscan.fit_predict(X_tsne)
print(f"Average silhouette score: {metrics.silhouette_score(X_tsne, cluster_labels_dbscan)}")


In [ ]:
fig, ax = plt.subplots(1)
mscatter(X_tsne[:, 0], X_tsne[:, 1], c=x_cluster_label_dbscan, s=20, ax=ax)
plt.show()

# 4) MA with AffinityPropagation clustering

In [ ]:
x_cluster_label_af.shape

In [ ]:
dstrb_sub = pd.DataFrame(X_tsne, index=subject_labels, columns=[0,1])
dstrb_sub['af_label'] = x_cluster_label_af

In [ ]:
dstrb_sub.head()

In [ ]:
clusters_centroids = af.cluster_centers_
print(f"cluster's centroids coordinates: \n {clusters_centroids}")

In [ ]:
dstrb_sub.loc[dstrb_sub.index[0]]

In [ ]:
central = []
for i in range(clusters_centroids.shape[0]):
    central.append(closest_distance(clusters_centroids[i], dstrb_sub.drop(['af_label'], axis=1)))
    print(f"Closest subject to centroid of cluster {i} is {central[i]}")

In [ ]:
arr = np.array([np.array([dstrb_sub[k][i] for k in dstrb_sub.columns[0:2]]) for i in range(len(dstrb_sub))])

color_dict = {0: 'blue', 1:'red', 2:'green', 3:'magenta'}
fig, ax = plt.subplots()

for g in np.unique([dstrb_sub.af_label]):
    ix = np.where(dstrb_sub.af_label == g)
    x = [arr[ix][k][0] for k in range(len(ix[0]))]
    y = [arr[ix][k][1] for k in range(len(ix[0]))]
    ax.scatter(x, y, c = color_dict[g], label = g)

for g in np.unique([dstrb_sub.af_label]):
    ax.scatter(clusters_centroids[g][0],
               clusters_centroids[g][1],
               color='black',
               marker='X',
               s=120)

plt.xlabel(f't-SNE dimension 1', fontsize=14)
plt.ylabel(f't-SNE dimension 2', fontsize=14)
plt.legend()
plt.show()

In [ ]:
arr = np.array([np.array([dstrb_sub[k][i] for k in dstrb_sub.columns[0:2]]) for i in range(len(dstrb_sub))])

color_dict = {0: 'blue', 1:'red', 2:'green', 3:'magenta'}
fig, ax = plt.subplots()

for g in np.unique([dstrb_sub.af_label]):
    ix = np.where(dstrb_sub.af_label == g)
    x = [arr[ix][k][0] for k in range(len(ix[0]))]
    y = [arr[ix][k][1] for k in range(len(ix[0]))]
    ax.scatter(x, y, c = color_dict[g], label = f"cluster {g}")

plt.xlabel(f'tSNE dimension 1', fontsize=14)
plt.ylabel(f'tSNE dimension 2', fontsize=14)
plt.legend(bbox_to_anchor=(1.28,1))
plt.show()

In [ ]:
print(len(dstrb_sub))

In [ ]:
clusters = []
for lab in np.unique([dstrb_sub.af_label]):
    cluster_df = dstrb_sub[dstrb_sub.af_label==lab]
    clusters.append(cluster_df)
for lab in np.unique([dstrb_sub.af_label]):
    assert(len(np.unique(list(clusters[lab].af_label)))==1)

### Creation of buckets dictionnary

In [ ]:
bucket_path = '/neurospin/dico/data/deep_folding/current/crops/CINGULATE/mask/sulcus_based/2mm/centered_combined/hcp/Rbuckets/'
suffix_path = '_normalized.bck'
buckets = {}
for sub in tqdm(list(subject_labels)):
    bucket = aims.read(os.path.join(bucket_path, str(sub) + suffix_path))
    bucket = dtx.convert.bucket_aims_to_ndarray(bucket[0])
    buckets[sub] = bucket

In [ ]:
clusters[0].index

In [ ]:
subjects = []
for lab in range(len(clusters)):
    subject = clusters[lab].index
    subjects.append(subject)

In [ ]:
buckets_c = []
for lab in range(len(subjects)):
    bucket_c = {k: v for k,v in buckets.items() if k in list(subjects[lab])}
    buckets_c.append(bucket_c)

In [ ]:
cld.draw(list(buckets_c[0].values())[0])

#### Alignement of the subjects to respective central subject

In [ ]:
aligned_buckets = []
aligned_rot = []
aligned_transl = []

for lab in range(len(subjects)):
    print(lab)
    buck, rot, trans = ma.align_buckets_by_ICP_batch(buckets_c[lab], central[lab])
    aligned_buckets.append(buck)
    aligned_rot.append(rot)
    aligned_transl.append(trans)

In [ ]:
for lab in range(len(subjects)):
    plt.hist(clusters[lab][0], alpha=0.5)
plt.show()

In [ ]:
for lab in range(len(subjects)):
    plt.hist(clusters[lab][1], alpha=0.5)
plt.show()

In [ ]:
def subj_count_extreme_coords(isodf, axis, min_coord, max_coord, num_coord=10):
    coord_values = np.linspace(min_coord, max_coord, num_coord)
    step = (coord_values[1] - coord_values[0])
    midpoint_min = coord_values[0] + step/2
    midpoint_max = coord_values[num_coord-1] - step/2
    coord_under = [elt for elt in isodf[axis] if elt<midpoint_min]
    coord_over = [elt for elt in isodf[axis] if elt>midpoint_max]
    num_subj_under = len(coord_under)
    num_subj_over = len(coord_over)
    return num_subj_under, num_subj_over

def get_MA_coords(isodf, axis, num_subj_threshold, num_coord=10):
    min_subj_coord = min(isodf[axis])
    max_subj_coord = max(isodf[axis])
    step = (max_subj_coord - min_subj_coord)/num_coord
    num_subj_under, num_subj_over = subj_count_extreme_coords(isodf, axis, min_subj_coord, max_subj_coord)
    while ((num_subj_under<num_subj_threshold)|(num_subj_over<num_subj_threshold)):
        step = (max_subj_coord - min_subj_coord)/num_coord
        if num_subj_under<num_subj_threshold:
            min_subj_coord = min_subj_coord + step
        if num_subj_over<num_subj_threshold:
            max_subj_coord = max_subj_coord - step
        num_subj_under, num_subj_over = subj_count_extreme_coords(isodf, axis, min_subj_coord, max_subj_coord)
    return np.linspace(min_subj_coord, max_subj_coord, num_coord)

In [ ]:
clusters[0].head()

In [ ]:
clusters_centroids

# Average on the first axis

In [ ]:
FWHM = [10., 10., 10., 10.]
SPAM_vols = [[], []]
shift = [[], []]
for ax in [0, 1]:
    for i in range(len(subjects)):
        vol_1, shift_1 = ma.moving_averages_tools.calc_one_MA_volume(
            buckets_dict=aligned_buckets[i], 
            distance_df=clusters[i],
            axis_n=ax,
            center=clusters_centroids[i][ax],
            FWHM=FWHM[i])
        SPAM_vols_1 = dict({clusters_centroids[i][ax]:vol_1})
        SPAM_vols[ax].append(SPAM_vols_1)
        shift[ax].append(shift_1)


In [ ]:
SPAM_meshes = [[], []]
thresholds = [[0.45, 0.44, 0.4, 0.4], [0.38, 0.42, 0.4, 0.42]]
threshold = 0.62

for ax in [0, 1]:
    for i in range(len(subjects)):
        SPAM_mesh_1 = {}

        for k, volume in tqdm(list(SPAM_vols[ax][i].items())[:]):
            SPAM_mesh_1[k]=dtx.convert.volume_to_mesh(
                            vol=volume,
                            gblur_sigma=0.0,
                            threshold=threshold,
                            deciReductionRate=0,
                            smoothRate=0.1)
        SPAM_meshes[ax].append(SPAM_mesh_1)


In [ ]:
fig = cld.draw(list(SPAM_vols[1][0].values())[0], th_min=0.2)
cld.draw(list(SPAM_meshes[1][0].values())[0], fig=fig)

In [ ]:
SPAM_meshes[0][1]

In [ ]:
for ax in [0, 1]:
    for i in range(len(subjects)):
        for _, mesh in tqdm(SPAM_meshes[ax][i].items()):
            aims.write(mesh, f"AF_cluster_ax{ax}_{i}.mesh")

# Find typical examples

In [ ]:
clusters[0].head()

In [ ]:
clusters[0].nsmallest(2,[0])

In [ ]:
def compute_distance(df_in, centroid):
    """Computes distance from centroid"""
    df_out = df_in
    df_out['distance'] = (df_out[0]-centroid[0])**2 + (df_out[1]-centroid[1])**2
    return df_out

In [ ]:
clusters[0] = compute_distance(clusters[0], clusters_centroids[0])

In [ ]:
neighbours = []
for i in range(len(subjects)):
    clusters[i] = compute_distance(clusters[i], clusters_centroids[i])
    neighbours.append(list(clusters[i].nsmallest(4,'distance').index))
print(neighbours)

In [ ]:
for idx, lab in enumerate(neighbours):
    for sub in lab:
        sub_bucket = dtx.convert.bucket_numpy_to_bucketMap_aims(buckets[sub])
        aims.write(sub_bucket, f"{idx}_{sub}.bck")